# CONSTANTS

In [64]:
# general imports
import pandas as pd
import numpy as np

# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

# Metrics
from sklearn.model_selection import cross_val_score

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV, ParameterGrid

# Feature selection
import librosa
from scipy.stats import skew, kurtosis
from scipy.signal import hilbert
from math import log10
import parselmouth
from maad import sound as suono
from maad import features as ft

# READ FUNCTION

In [65]:
csvTotale = pd.read_csv('./data/development.csv', header=0, index_col=0).drop(columns=['sampling_rate'])
csvTotale['gender'] = csvTotale['gender'].map(lambda x: 0 if x=='male' else 1)
csvTotale['tempo'] = csvTotale['tempo'].map(lambda x: float(x[1:-1]))
csvTotale['path'] = csvTotale['path'].map(lambda x: x.split('/')[-1])

In [66]:
csvEval = pd.read_csv('./data/evaluation.csv', header=0, index_col=0).drop(columns=['sampling_rate'])
csvEval['gender'] = csvEval['gender'].map(lambda x: 0 if x=='male' else 1)
csvEval['tempo'] = csvEval['tempo'].map(lambda x: float(x[1:-1]))
csvEval['path'] = csvEval['path'].map(lambda x: x.split('/')[-1])

# Baselines by default

In [ ]:
(cross_val_score(HistGradientBoostingRegressor(categorical_features=csvTotale.drop(columns=['path', 'age']).dtypes == 'object'), 
                csvTotale.drop(columns=['path', 'age']), 
                csvTotale['age'], cv=10, scoring='neg_root_mean_squared_error').mean(),
 cross_val_score(make_pipeline(StandardScaler(), MLPRegressor(max_iter=1000)), 
                csvTotale.drop(columns=['path', 'age', 'ethnicity']), 
                csvTotale['age'], cv=10, scoring='neg_root_mean_squared_error').mean())

# BASELINE = -7.24, -8.18

In [69]:
audioDevelopment = {file: librosa.load('./data/audios_development/'+file, sr=22050)[0] for file in csvTotale['path']}

# FUNCTION TO EXTRACT FURTHER FEATURES

In [70]:
def getMFCC(audio):
    numFcc = 35
    return pd.Series(librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=numFcc).mean(axis=1), index=[f'mfcc{i}' for i in range(numFcc)])

In [71]:
def getSpectralEnergy(audio):
    S = librosa.stft(audio)
    freqs = librosa.fft_frequencies(sr=22050)
    return pd.Series([np.sum(np.abs(S[(freqs >= 250) & (freqs <= 650)])**2), np.sum(np.abs(S[(freqs >= 1000) & (freqs <= 8000)])**2)],
                        index=['spectralEnergy250-650', 'spectralEnergy1000-8000'])

In [72]:
def computeParselMouthStats(audio):
    sound = parselmouth.Sound(audio)
    pitch = sound.to_pitch()
    info = str(pitch.info()).split('\n')
    return pd.Series([pitch.count_voiced_frames(), pitch.get_mean_absolute_slope(), pitch.xmax-pitch.xmin, 
                      pitch.n_frames, *[float(info[15+i].split('=')[2].lstrip().split(' =')[0].split()[0]) for i in range(0, 5)],
                      *[float(info[21+i].split('=')[2].lstrip().split(' =')[0].split()[0]) for i in range(0, 3)]],
                     index=['nVoicedFrames', 'meanAbsoluteSlope', 'duration', 'nFrames', 
                            'q10', 'q16', 'q50', 'q84', 'q90', '84-median', 'median-16', '90-10']
                     )

In [73]:
def computeMath(audio):
    return pd.Series([skew(audio), kurtosis(audio), np.mean(np.abs(hilbert(audio))), np.mean(np.abs(np.fft.fft(audio)))],
                     index=['skew', 'kurtosis', 'hilbertMean', 'fftMean'])

In [74]:
def computSNR(audio):
    return pd.Series([suono.temporal_snr(audio)[-1]], index=['temporalSNR'])

In [75]:
def computeTemporalMedia(audio):
    return pd.Series([ft.temporal_median(audio)], index=['temporalMedian'])

In [76]:
def computeAllFeatures(audio):
    return pd.Series(ft.all_temporal_features(audio, fs=22050, nperseg=256).values[0],
                     index=['sm', 'sv', 'ss', 'sk', 'Time 5%', "Time 25%", "Time 50%", "Time 75%", "Time 95%  ", 
                            "zcr", "duration_5", "duration_90"])

In [77]:
def computePeakFrequency(audio):
    return pd.Series(ft.peak_frequency(audio, fs=22050, nperseg=256, amp=True), index=['peakFrequency', 'peakFrequencyAmp'])

In [78]:
def computeEntropy(audio):
    return pd.Series(ft.temporal_entropy(audio), index=['entropy'])

In [79]:
# NO BUENO JUST NOISE
def computeMaadStats(audio):
    Sxx_power, tn, fn, _ = suono.spectrogram(audio, 22050)      
    return pd.Series([ft.number_of_peaks(Sxx_power, fn=fn, nperseg=256), 
                      ft.bioacoustics_index(Sxx_power, fn=fn, flim=(100, 3000)),
                      ft.acoustic_diversity_index(Sxx_power, fn=fn, fmin=80, fmax=3000),
                      ft.acoustic_eveness_index(Sxx_power, fn=fn, fmin=80, fmax=3000),
                      ],
                    index=['nPeaks', 'bioIndex', 'acousticDiversity', 'acousticEvenness'])

# COMPUTE ALL FEATURES

In [80]:
def computeMetrics(audios):
    return pd.DataFrame({file: pd.concat([
            getMFCC(audios[file]),
            getSpectralEnergy(audios[file]),
            computeParselMouthStats(audios[file]),
            computeMath(audios[file]),
            computSNR(audios[file]),
            computeTemporalMedia(audios[file]),
            computeAllFeatures(audios[file]),
            computePeakFrequency(audios[file]),
            computeEntropy(audios[file]),   
            ], axis=0)
        for file in audios}).T

In [81]:
metrics = pd.concat([computeMetrics(audioDevelopment), 
                    csvTotale.set_index('path')[['hnr', 'shimmer', 'jitter', 'gender', 'max_pitch', 'mean_pitch', 'min_pitch']]], axis=1)

metrics['hnr']= metrics['hnr'].map(lambda x: 10*log10(np.abs(x)))

# CROSS VAL SCORE AND IMPORTANCE

In [ ]:
cross_val_score(HistGradientBoostingRegressor(), metrics,
                 csvTotale['age'], cv=15, scoring='neg_root_mean_squared_error').mean()

In [ ]:
for item, imp in sorted(zip(list(metrics.columns)+['age'], 
                            pd.concat([metrics, csvTotale.set_index(csvTotale['path'])['age']], axis=1).corr('spearman')['age']), key=lambda x: abs(x[1]), reverse=True):
    print(f'{item}: {imp}')

# READ AUDIO EVAL

In [84]:
audioEval = {file: librosa.load('./data/audios_evaluation/'+file, sr=22050)[0] for file in csvEval['path']}

# COMPUTE STATS AND REGRESSION

In [85]:
metricsEval = pd.concat([computeMetrics(audioEval), 
                    csvEval.set_index('path')[['hnr', 'shimmer', 'jitter', 'gender', 'max_pitch', 'mean_pitch', 'min_pitch']]], axis=1)
metricsEval['hnr']= metricsEval['hnr'].map(lambda x: 20*log10(np.abs(x)))

In [86]:
pd.Series(HistGradientBoostingRegressor(warm_start=True).fit(metrics, csvTotale['age']).predict(metricsEval),
          name='Predicted',
          index=csvEval.index).to_csv('finalPredict.csv', index_label='Id')

# OTHER MODELS

In [ ]:
-cross_val_score(RandomForestRegressor(n_jobs=-1), metrics, csvTotale['age'], cv=15, scoring='neg_root_mean_squared_error').mean()

In [ ]:
-cross_val_score(make_pipeline(StandardScaler(), MLPRegressor(max_iter=int(1e4))), metrics, csvTotale['age'], cv=15, scoring='neg_root_mean_squared_error').mean()

In [ ]:
-cross_val_score(make_pipeline(StandardScaler(), SVR()), metrics, csvTotale['age'], cv=15, scoring='neg_root_mean_squared_error').mean()

# BASELINES

In [ ]:
pd.Series(np.zeros(csvEval.shape[0])+
          (csvTotale['age'].mean()+csvTotale['age'].median())/2,  
          name='Predicted').to_csv('Naive-Baseline.csv', header=True, index_label='Id')

In [ ]:
lista = list(
    map(lambda x: [x, -cross_val_score(DecisionTreeRegressor(**x), metrics, csvTotale['age'], 
                                       cv=15, scoring='neg_root_mean_squared_error').mean()],
        ParameterGrid({
            'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
            'splitter': ['best', 'random'],
            'max_features': ['sqrt', 'log2'],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_depth': [10, 20, 30, None]
        })
))

In [ ]:
pd.Series(DecisionTreeRegressor(**(sorted(lista, key=lambda x: x[1])[0][0]))
          .fit(metrics, csvTotale['age'])
          .predict(metricsEval),
          name='Predicted').to_csv('Tree-Baseline.csv', header=True, index_label='Id')

In [ ]:
tempScaler = StandardScaler()

pd.Series((SVR().fit(tempScaler.fit_transform(metrics), csvTotale['age']).predict(tempScaler.transform(metricsEval)) +
          RandomForestRegressor(n_jobs=-1).fit(metrics, csvTotale['age']).predict(metricsEval))/2,
            name='Predicted').to_csv('Ensemble.csv', header=True, index_label='Id')

In [ ]:
-(cross_val_score(make_pipeline(StandardScaler(), SVR()), metrics, csvTotale['age'], cv=15, scoring='neg_root_mean_squared_error').mean()+
cross_val_score(RandomForestRegressor(n_jobs=-1), metrics, csvTotale['age'], cv=15, scoring='neg_root_mean_squared_error').mean())/2

# GRID SEARCH

In [ ]:
param_grid = {
    'loss':['squared_error', 'absolute_error', 'poisson', 'quantile'],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_iter': [100, 200, 300],
    'max_leaf_nodes': [31, 63, 127],
    'min_samples_leaf': [20, 50, 100]
}

grid_search = GridSearchCV(estimator=HistGradientBoostingRegressor(), param_grid=param_grid, 
                           cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1)

grid_search.fit(metrics, csvTotale['age'])

print("Best parameters found: ", grid_search.best_params_)
print("Best RMSE score: ", -grid_search.best_score_)